In [84]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, assemble 
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.extensions import UnitaryGate
import math
# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-09-27 14:22:13,882: Credentials are already in use. The existing account in the session will be replaced.


In [85]:
def decimalToBinary(n):
    return "{0:b}".format(int(n))
    

In [87]:
input_list = [1, 5, 4, 2]
m = int(math.ceil(math.log(max(input_list), 2)))
binary_list = []
for x in input_list:
    binary_list.append(decimalToBinary(x))
print(binary_list)
n = 3
qc = QuantumCircuit(n)

matrix_4qubits = np.asmatrix(np.eye(16), dtype=int)
matrix_4qubits[5, 5] = -1
matrix_4qubits[10, 10] = -1
matrix_5qubits = np.asmatrix(np.eye(32), dtype=int)
matrix_5qubits[21, 21] = -1
matrix_5qubits[10, 10] = -1
matrix_3qubits = [[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,-1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]]
matrix_2qubits = np.asmatrix(np.eye(4), dtype=int)
winning_indices = []
print(m)
if m == 5:
    winning_indices.append(input_list.index(10))
    winning_indices.append(input_list.index(21))
if m == 4:
    winning_indices.append(input_list.index(5))
    winning_indices.append(input_list.index(10))
if m == 3:
    winning_indices.append(input_list.index(2))
    winning_indices.append(input_list.index(5))
if m == 2:
    winning_indices.append(input_list.index(1))
    winning_indices.append(input_list.index(2))
if m < 2:
    print("Please enter a value >= 1")

print(winning_indices)
matrix_3qubits = np.asmatrix(np.eye(8), dtype=int)
for i in winning_indices:
    matrix_3qubits[i, i] = -1
print(matrix_3qubits)

oracle_z = UnitaryGate(matrix_3qubits, "U$_\omega$")

qc.unitary(oracle_z,list(range(n)))
oracle_ex3 = qc.to_gate()


['1', '101', '100', '10']
3
[3, 1]


TypeError: list indices must be integers or slices, not tuple

In [ ]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

In [ ]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [ ]:
n = 3
grover_circuit = QuantumCircuit(n)
grover_circuit = initialize_s(grover_circuit, list(range(n)))
grover_circuit.append(oracle_ex3, list(range(n)))
grover_circuit.append(diffuser(n), [0,1,2])
grover_circuit.measure_all()
grover_circuit.draw()

In [ ]:
aer_sim = Aer.get_backend('aer_simulator')
transpiled_grover_circuit = transpile(grover_circuit, aer_sim)
qobj = assemble(transpiled_grover_circuit)
results = aer_sim.run(qobj).result()
counts = results.get_counts()
plot_histogram(counts)